# Comet

![](https://user-images.githubusercontent.com/7529846/230328046-a8b18c51-12e3-4617-9b39-97614a571a2d.png)

In this guide we will demonstrate how to track your Langchain Experiments, Evaluation Metrics, and LLM Sessions with [Comet](https://www.comet.com/site/?utm_source=langchain&utm_medium=referral&utm_campaign=comet_notebook).  

<a target="_blank" href="https://colab.research.google.com/github/hwchase17/langchain/blob/master/docs/ecosystem/comet_tracking.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Example Project:** [Comet with LangChain](https://www.comet.com/lothiraldan/langchain-integration-beta/prompts?utm_source=langchain&utm_medium=referral&utm_campaign=comet_notebook)

<img width="1280" alt="comet-langchain" src="https://user-images.githubusercontent.com/7529846/230326720-a9711435-9c6f-4edb-a707-94b67271ab25.png">


### Install Comet and Dependencies

In [ ]:
%pip install -U comet_llm "langchain>=0.0.162" openai google-search-results spacy textstat pandas

import sys

!{sys.executable} -m spacy download en_core_web_sm

### Initialize Comet and Set your Credentials

You can grab your [Comet API Key here](https://www.comet.com/signup?utm_source=langchain&utm_medium=referral&utm_campaign=comet_notebook):

In [ ]:
import comet_llm

# os.environ['COMET_API_KEY'] = YOUR-COMET-API-KEY
# os.environ['COMET_PROJECT_NAME'] = YOUR-PROJECT-NAME

### Set OpenAI and SerpAPI credentials

You will need an [OpenAI API Key](https://platform.openai.com/account/api-keys) and a [SerpAPI API Key](https://serpapi.com/dashboard) to run the following examples

In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = "..."
# os.environ["OPENAI_ORGANIZATION"] = "..."
# os.environ["SERPAPI_API_KEY"] = "..."

### Define the beta-version of the Langchain callback

In [ ]:
from langchain.callbacks.tracers.base import BaseTracer


def get_run_type(run):
    if isinstance(run.run_type, str):
        return run.run_type
    elif hasattr(run.run_type, "value"):
        return run.run_type.value
    else:
        return str(run.run_type)


class CometTracer(BaseTracer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.span_map = {}

    def _persist_run(self, run) -> None:
        comet_llm.end_chain(outputs=run.outputs)

    def _start_trace(self, run) -> None:
        if not run.parent_run_id:
            # This is the first run, which maps to a chain
            comet_llm.start_chain(inputs=run.inputs)
        else:
            span = comet_llm.Span(
                inputs=run.inputs,
                category=get_run_type(run),
                metadata=run.extra,
                name=run.name,
            ).__enter__()
            self.span_map[run.id] = span
        super()._start_trace(run)

    def _end_trace(self, run) -> None:
        if not run.parent_run_id:
            pass
            # Langchain will call _persist_run for us
        else:
            span = self.span_map[run.id]
            span.set_outputs(outputs=run.outputs)
            span.__exit__(None, None, None)
        super()._end_trace(run)

### Scenario 1: Using just an LLM

In [ ]:
from datetime import datetime

from langchain.llms import OpenAI

comet_tracer = CometTracer()
callbacks = [comet_tracer]
llm = OpenAI(temperature=0.9, callbacks=callbacks, verbose=True)

for prompt in ["Tell me a joke", "Tell me a poem", "Tell me a fact"]:
    llm_result = llm.generate([prompt], callbacks=[comet_tracer])
    print("LLM result", llm_result)

### Scenario 2: Using an LLM in a Chain

In [ ]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

tracer = CometTracer()
callbacks = [tracer]

llm = OpenAI(temperature=0.9, callbacks=callbacks, verbose=True)

template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(
    llm=llm, prompt=prompt_template, callbacks=callbacks, verbose=True
)

test_prompts = [{"title": "Documentary about Bigfoot in Paris"}]
print(synopsis_chain.apply(test_prompts))

### Scenario 3: Using An Agent with Tools 

In [ ]:
from langchain.agents import initialize_agent, load_tools
from langchain.llms import OpenAI


tracer = CometTracer()
callbacks = [tracer]

llm = OpenAI(temperature=0.9, callbacks=callbacks, verbose=True)

tools = load_tools(["serpapi", "llm-math"], llm=llm, callbacks=callbacks, verbose=True)
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    callbacks=callbacks,
    verbose=True,
)
agent.run(
    "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?",
    callbacks=callbacks,
)